## Q1.

Add methods `__iter__` to your project Time Series class to iterate over values, a method `itertimes` to iterate over times, a method `itervalues` to iterate over values, and a method `iteritems` to iterate over time-value pairs. (This is a similar interface to python dictionaries). To test these, check both the types of the results and the answers you expect.

In [1]:
#your code here
# Added to our TimeSeries class:
    def __iter__(self):
        return iter(self._values)

    def iteritems(self):
        return zip(self._times,self._values)

    def itervalues(self):
        return iter(self._values)

    def itertimes(self):
        return iter(self._times)

## Q2.

An online mean and standard deviation algorithm.

Below is a function to generate a potentially infinite stream of 1-D data.

In [1]:
from random import normalvariate, random
from itertools import count
def make_data(m, stop=None):
    for _ in count():
        if stop and _ > stop:
            break
        yield 1.0e09 + normalvariate(0, m*random() )
        

Here is an implementation of an online mean algorithm..see http://www.johndcook.com/blog/standard_deviation/ and the link to http://www.johndcook.com/blog/2008/09/26/comparing-three-methods-of-computing-standard-deviation/ in-between. (Convince yourselves of the formulas...)

In [2]:
def online_mean(iterator):
    n = 0
    mu = 0
    for value in iterator:
        n += 1
        delta = value - mu
        mu = mu + delta/n
        yield mu

We use out generator functions to implement iterators:

In [32]:
g = make_data(5, 10)
list(g)

[1000000000.6633602,
 999999999.1531693,
 999999998.0347095,
 999999997.3845587,
 1000000000.1360068,
 1000000003.7602679,
 999999997.8984545,
 999999998.4067945,
 999999995.1473001,
 1000000004.3249445,
 999999998.5533131]

In [34]:
g = online_mean(make_data(5, 100))
print(type(g))
list(g)[:5]

<class 'generator'>


[1000000005.122938,
 1000000002.5397222,
 1000000001.3473922,
 1000000002.9736841,
 1000000002.3926823]

### 2.1

Implement the standard deviation algorithm as a generator function as

```python
def online_mean_dev(iterator):
    BLA BLA
    if n > 1:
        stddev = math.sqrt(dev_accum/(n-1))
        yield (n, value, mu, stddev)
```

In [67]:
# your code here
import math
def online_mean_dev(iterator):
    mu = next(iterator)
    dev_accum = 0
    n=0
    
    for value in iterator:
        n+=1
        old_mu = mu
        mu = old_mu + (value - old_mu)/n
        dev_accum = dev_accum + (value - old_mu)*(value - mu)
    
        if n > 1:
            stddev = math.sqrt(dev_accum/(n-1))
            yield (n, value, mu, stddev)

Here we make 100000 element data, and run this iterator on it (imagine running this on a time-series being slowly read from disk

In [68]:
data_with_stats = online_mean_dev(make_data(5, 100000))

In [66]:
list(data_with_stats)[9999:10004]

[(10001, 999999995.1955078, 999999999.9657292, 2.904659908960276),
 (10002, 1000000004.9114842, 999999999.9662237, 2.9049356492541385),
 (10003, 1000000000.2235223, 999999999.9662495, 2.9047915670843594),
 (10004, 999999999.022243, 999999999.966155, 2.9046617013067224),
 (10005, 999999996.969372, 999999999.9658555, 2.9046710403154172)]

## Q3.

Let's do Anomaly detection. Write a routine `is_ok`:

```python
def is_ok(level, t)
```

which takes a tuple like the one yielded by your code above and returns True if the value is inbetween `level`-$\sigma$ of the mean.

In [69]:
#your code here
def is_ok(level, t):
    n, value, mu, stddev = t
    above = (value >= (mu - level*stddev))
    below = (value <= (mu + level*stddev))
    return (above & below)

We use this function to create a predicate passed through to `itertools.filterfalse` which is then used to obtain an iterator on the anomalies.

In [70]:
from itertools import filterfalse
pred = lambda t: is_ok(5, t)
anomalies = filterfalse(pred, data_with_stats)

We materialize the anomalies...

In [71]:
list(anomalies)#materialize

[(1217, 1000000016.1595209, 1000000000.1549648, 2.8917077035400975),
 (1348, 999999982.761378, 1000000000.1251444, 2.937724352965173),
 (6562, 999999983.7535243, 999999999.9877862, 2.9089096579826323),
 (7323, 1000000014.7159464, 999999999.9954739, 2.9233229696920486),
 (7592, 999999983.7700639, 999999999.9895842, 2.923596534425622),
 (7763, 999999984.1318893, 999999999.9914607, 2.9260590362678283),
 (8855, 1000000014.5991789, 999999999.9965973, 2.9157340703520203),
 (10022, 999999985.2586726, 999999999.9938676, 2.9331859420690214),
 (12422, 999999984.8763034, 1000000000.0027957, 2.932280894094167),
 (13012, 1000000016.3651265, 1000000000.0054185, 2.932499917539923),
 (21939, 1000000015.6092161, 1000000000.0171708, 2.895917155814586),
 (22643, 1000000016.8827181, 1000000000.010105, 2.8984798313114335),
 (24353, 1000000014.7571688, 1000000000.0127407, 2.899329179735781),
 (27719, 999999984.6054496, 1000000000.0160838, 2.8936640046860056),
 (39421, 1000000014.5171062, 1000000000.0109216,

## To think of, but not hand in

What kinds of anomalies will this algorithm pick up? What kinds would a shorter "window" of anomaly detection, like 100 points around the time in question pick? How might you create an algorithm which does window based averaging? (hint: the window size is small compared to the time series size). 

Finally think a bit of how you might implement all of this in a production environment..remember that data streaming in might get backed up when you handle an anomaly.

(Some inspiration might accrue if you look at the docs for `collections.deque`).

### The algorithm picks up values that are more or less than the number of standard deviations passed to the algorithm, where the standard deviation is a running calculation based on all of your data up until that point.  One limitation would be if your mean changes over time due to the characteristics of your incoming data.  In this case, you might want to see if a value is anomalous around the time window (for example, to detect a large measurement error in a time series that has a mean that changes over time).  In this case, you might want to run over the values in the iterator and do the comparison with a standard deviation for that window rather than the standard deviation of all the data up until that point.  
### In a production environment, you might want to have some data storage between the data streaming and the anomaly detection so that you can continue to run the anomaly detection while the data streaming is backed up.  